In [48]:
import psycopg2
import pandas as pd

In [47]:
#connect to postgreSQL database (asmt2_de)
conn = psycopg2.connect(
    host="localhost",
    port=5432,
    database="asmt2_de",
    user="postgres",
    password="2358"
)

#create a cursor to execute SQL commands
cur = conn.cursor()

print("Success!")

Success!


In [49]:
#create the database structure
cur.execute("""
-- drop in reverse order to avoid foreign key issues
DROP TABLE IF EXISTS companies CASCADE;
DROP TABLE IF EXISTS ceos CASCADE;
DROP TABLE IF EXISTS cities CASCADE;
DROP TABLE IF EXISTS states CASCADE;
DROP TABLE IF EXISTS countries CASCADE;

-- create COUNTRIES TABLE
CREATE TABLE countries (
    country_id SERIAL PRIMARY KEY,
    country_name VARCHAR(100) UNIQUE NOT NULL
);

-- create STATES TABLE
CREATE TABLE states (
    state_id SERIAL PRIMARY KEY,
    state_name VARCHAR(100) UNIQUE,
    country_id INT NOT NULL REFERENCES countries(country_id)
);

-- create CITIES TABLE
CREATE TABLE cities (
    city_id SERIAL PRIMARY KEY,
    city_name VARCHAR(100) NOT NULL,
    state_id INT REFERENCES states(state_id),
    country_id INT NOT NULL REFERENCES countries(country_id)
);

-- create CEOs TABLE
CREATE TABLE ceos (
    ceo_id SERIAL PRIMARY KEY,
    first_name VARCHAR(50) NOT NULL,
    last_name VARCHAR(50) NOT NULL,
    birth_year INT CHECK (birth_year > 1900),
    city_id INT NOT NULL REFERENCES cities(city_id),
    became_ceo_year INT CHECK (became_ceo_year > 1900)
);

-- create COMPANIES TABLE
CREATE TABLE companies (
    company_id SERIAL PRIMARY KEY,
    company_name VARCHAR(100) UNIQUE NOT NULL,
    founding_year INT CHECK (founding_year > 1800),
    ceo_id INT NOT NULL REFERENCES ceos(ceo_id),
    hq_city_id INT NOT NULL REFERENCES cities(city_id)
);
""")

conn.commit()

print("all tables created successfully with referential integrity!")

all tables created successfully with referential integrity!


In [50]:
#populate geographic data with full referential integrity
cur.execute("""
-- insert countries
INSERT INTO countries (country_name)
VALUES
('USA'),
('India'),
('Japan'),
('China'),
('France'),
('South Korea');

-- insert states (everywhere but Japan)
INSERT INTO states (state_name, country_id)
VALUES
('Alabama', (SELECT country_id FROM countries WHERE country_name = 'USA')),
('California', (SELECT country_id FROM countries WHERE country_name = 'USA')),
('New York', (SELECT country_id FROM countries WHERE country_name = 'USA')),
('Washington', (SELECT country_id FROM countries WHERE country_name = 'USA')),
('Michigan', (SELECT country_id FROM countries WHERE country_name = 'USA')),
('Arkansas', (SELECT country_id FROM countries WHERE country_name = 'USA')),
('Tennessee', (SELECT country_id FROM countries WHERE country_name = 'USA')),
('Andhra-Pradesh', (SELECT country_id FROM countries WHERE country_name = 'India')),
('Gyeonggi-do', (SELECT country_id FROM countries WHERE country_name = 'South Korea')),
('Zhejiang', (SELECT country_id FROM countries WHERE country_name = 'China')),
('Illes-et-Villaine', (SELECT country_id FROM countries WHERE country_name = 'France'));

-- insert cities (CEO birthplaces and company HQs)
INSERT INTO cities (city_name, state_id, country_id)
VALUES
('Mobile', (SELECT state_id FROM states WHERE state_name = 'Alabama'),
         (SELECT country_id FROM countries WHERE country_name = 'USA')),

('Cupertino', (SELECT state_id FROM states WHERE state_name = 'California'),
         (SELECT country_id FROM countries WHERE country_name = 'USA')),

('Armonk', (SELECT state_id FROM states WHERE state_name = 'New York'),
         (SELECT country_id FROM countries WHERE country_name = 'USA')),

('Scarsdale', (SELECT state_id FROM states WHERE state_name = 'New York'),
         (SELECT country_id FROM countries WHERE country_name = 'USA')),

('Seattle', (SELECT state_id FROM states WHERE state_name = 'Washington'),
         (SELECT country_id FROM countries WHERE country_name = 'USA')),

('Royal Oak', (SELECT state_id FROM states WHERE state_name = 'Michigan'),
         (SELECT country_id FROM countries WHERE country_name = 'USA')),

('Detroit', (SELECT state_id FROM states WHERE state_name = 'Michigan'),
         (SELECT country_id FROM countries WHERE country_name = 'USA')),

('Bentonville', (SELECT state_id FROM states WHERE state_name = 'Arkansas'),
         (SELECT country_id FROM countries WHERE country_name = 'USA')),

('Memphis', (SELECT state_id FROM states WHERE state_name = 'Tennessee'),
         (SELECT country_id FROM countries WHERE country_name = 'USA')),

('West Godavari', (SELECT state_id FROM states WHERE state_name = 'Andhra-Pradesh'),
         (SELECT country_id FROM countries WHERE country_name = 'India')),

('Tokyo', NULL,
         (SELECT country_id FROM countries WHERE country_name = 'Japan')),

('Kumamoto', NULL,
         (SELECT country_id FROM countries WHERE country_name = 'Japan')),

('Shanghai', NULL,
         (SELECT country_id FROM countries WHERE country_name = 'China')),

('Hangzhou', (SELECT state_id FROM states WHERE state_name = 'Zhejiang'),
         (SELECT country_id FROM countries WHERE country_name = 'China')),

('Rennes', (SELECT state_id FROM states WHERE state_name = 'Illes-et-Villaine'),
         (SELECT country_id FROM countries WHERE country_name = 'France')),

('Paris', NULL,
         (SELECT country_id FROM countries WHERE country_name = 'France')),

('Seoul', NULL,
         (SELECT country_id FROM countries WHERE country_name = 'South Korea')),

('Suwon', (SELECT state_id FROM states WHERE state_name = 'Gyeonggi-do'),
         (SELECT country_id FROM countries WHERE country_name = 'South Korea'));
""")

conn.commit()
print("All countries, states, and cities inserted successfully with subquery encoding!")


All countries, states, and cities inserted successfully with subquery encoding!


In [51]:
#insert CEO data
cur.execute("""
INSERT INTO ceos (first_name, last_name, birth_year, city_id, became_ceo_year)
VALUES
('Tim', 'Cook', 1960,
    (SELECT city_id FROM cities
     WHERE city_name = 'Mobile'
        AND state_id = (SELECT state_id FROM states WHERE state_name = 'Alabama')
        AND country_id = (SELECT country_id FROM countries WHERE country_name = 'USA')),
     2011),

('Arvind', 'Krishna', 1962,
    (SELECT city_id FROM cities
     WHERE city_name = 'West Godavari'
        AND state_id = (SELECT state_id FROM states WHERE state_name = 'Andhra-Pradesh')
        AND country_id = (SELECT country_id FROM countries WHERE country_name = 'India')),
     2020),

('Andy','Jassy',1968,
    (SELECT city_id FROM cities
     WHERE city_name = 'Scarsdale'
       AND state_id = (SELECT state_id FROM states WHERE state_name = 'New York')
       AND country_id = (SELECT country_id FROM countries WHERE country_name = 'USA')),
    2021),

('Mary','Barra',1961,
    (SELECT city_id FROM cities
     WHERE city_name = 'Royal Oak'
       AND state_id = (SELECT state_id FROM states WHERE state_name = 'Michigan')
       AND country_id = (SELECT country_id FROM countries WHERE country_name = 'USA')),
    2014),

('Kenichiro','Yoshida',1959,
    (SELECT city_id FROM cities
     WHERE city_name = 'Kumamoto'
       AND state_id IS NULL
       AND country_id = (SELECT country_id FROM countries WHERE country_name = 'Japan')),
    2018),

('Daniel','Zhang',1972,
    (SELECT city_id FROM cities
     WHERE city_name = 'Shanghai'
       AND state_id IS NULL
       AND country_id=(SELECT country_id FROM countries WHERE country_name= 'China')),
    2015),

('François-Henri','Pinault',1962,
    (SELECT city_id FROM cities
     WHERE city_name = 'Rennes'
       AND state_id=(SELECT state_id FROM states WHERE state_name = 'Illes-et-Villaine')
       AND country_id=(SELECT country_id FROM countries WHERE country_name = 'France')),
    2005),

('Jong-hee','Han',1962,
    (SELECT city_id FROM cities
     WHERE city_name = 'Seoul'
       AND state_id IS NULL
       AND country_id=(SELECT country_id FROM countries WHERE country_name = 'South Korea')),
    2022),

('Doug','McMillon',1966,
    (SELECT city_id FROM cities
     WHERE city_name = 'Memphis'
       AND state_id=(SELECT state_id FROM states WHERE state_name = 'Tennessee')
       AND country_id=(SELECT country_id FROM countries WHERE country_name = 'USA')),
    2014);
""")
conn.commit()
print("CEOs and their info inserted successfully!")



CEOs and their info inserted successfully!


In [52]:
#insert companies info
cur.execute("""
INSERT INTO companies (company_name, founding_year, ceo_id, hq_city_id)
VALUES
('Apple, Inc.', 1976,
    (SELECT ceo_id FROM ceos WHERE first_name = 'Tim' AND last_name = 'Cook' AND birth_year = 1960),
    (SELECT city_id FROM cities
     WHERE city_name = 'Cupertino'
       AND state_id = (SELECT state_id FROM states WHERE state_name = 'California')
       AND country_id = (SELECT country_id FROM countries WHERE country_name = 'USA'))),

('IBM', 1911,
    (SELECT ceo_id FROM ceos WHERE first_name = 'Arvind' AND last_name = 'Krishna' AND birth_year = 1962),
    (SELECT city_id FROM cities
     WHERE city_name = 'Armonk'
       AND state_id = (SELECT state_id FROM states WHERE state_name = 'New York')
       AND country_id = (SELECT country_id FROM countries WHERE country_name = 'USA'))),

('Amazon', 1994,
    (SELECT ceo_id FROM ceos WHERE first_name='Andy' AND last_name='Jassy' AND birth_year = 1968),
    (SELECT city_id FROM cities
     WHERE city_name = 'Seattle'
       AND state_id = (SELECT state_id FROM states WHERE state_name = 'Washington')
       AND country_id = (SELECT country_id FROM countries WHERE country_name = 'USA'))),

('General Motors', 1908,
    (SELECT ceo_id FROM ceos WHERE first_name = 'Mary' AND last_name = 'Barra' AND birth_year = 1961),
    (SELECT city_id FROM cities
     WHERE city_name = 'Detroit'
       AND state_id = (SELECT state_id FROM states WHERE state_name = 'Michigan')
       AND country_id = (SELECT country_id FROM countries WHERE country_name = 'USA'))),

('Sony Corporation', 1946,
    (SELECT ceo_id FROM ceos WHERE first_name = 'Kenichiro' AND last_name = 'Yoshida' AND birth_year = 1959),
    (SELECT city_id FROM cities
     WHERE city_name = 'Tokyo'
       AND state_id IS NULL
       AND country_id = (SELECT country_id FROM countries WHERE country_name = 'Japan'))),

('Alibaba Group', 1999,
    (SELECT ceo_id FROM ceos WHERE first_name = 'Daniel' AND last_name = 'Zhang' AND birth_year = 1972),
    (SELECT city_id FROM cities
     WHERE city_name = 'Hangzhou'
       AND state_id = (SELECT state_id FROM states WHERE state_name = 'Zhejiang')
       AND country_id = (SELECT country_id FROM countries WHERE country_name = 'China'))),

('Kering SA', 1963,
    (SELECT ceo_id FROM ceos WHERE first_name = 'François-Henri' AND last_name = 'Pinault' AND birth_year = 1962),
    (SELECT city_id FROM cities
     WHERE city_name = 'Paris'
       AND state_id IS NULL
       AND country_id = (SELECT country_id FROM countries WHERE country_name = 'France'))),

('Samsung Electronics', 1969,
    (SELECT ceo_id FROM ceos WHERE first_name = 'Jong-hee' AND last_name = 'Han' AND birth_year = 1962),
    (SELECT city_id FROM cities
     WHERE city_name = 'Suwon'
       AND state_id=(SELECT state_id FROM states WHERE state_name = 'Gyeonggi-do')
       AND country_id=(SELECT country_id FROM countries WHERE country_name = 'South Korea'))),

('Walmart', 1962,
    (SELECT ceo_id FROM ceos WHERE first_name = 'Doug' AND last_name = 'McMillon' AND birth_year = 1966),
    (SELECT city_id FROM cities
     WHERE city_name = 'Bentonville'
       AND state_id = (SELECT state_id FROM states WHERE state_name = 'Arkansas')
       AND country_id = (SELECT country_id FROM countries WHERE country_name = 'USA')));
""")
conn.commit()
print("Companies and their info inserted successfully!")


Companies and their info inserted successfully!


In [57]:
#1. What’s the age of the company whose CEO was born in the state of Tennessee?
#joins CEO+birth city+state; filter for TN; compute 2025 - founding year
pd.read_sql("""
SELECT comp.company_name,
       2025 - comp.founding_year AS company_age
FROM companies comp
JOIN ceos ce ON comp.ceo_id = ce.ceo_id
JOIN cities ci ON ce.city_id = ci.city_id
JOIN states st ON ci.state_id = st.state_id
WHERE st.state_name = 'Tennessee';
""", conn)

/var/folders/t2/9wrz89jj3tnbfwphbvxvcl8r0000gn/T/ipykernel_27279/92093104.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("""


,company_name,company_age
0,Walmart,63


In [58]:
#2. What are the full names of CEOs of companies founded in the USA before 1965?
pd.read_sql("""
SELECT ce.first_name || ' ' || ce.last_name AS ceo_name,
       comp.company_name,
       comp.founding_year
FROM companies comp
JOIN ceos ce      ON comp.ceo_id = ce.ceo_id
JOIN cities hc    ON comp.hq_city_id = hc.city_id
JOIN countries co ON hc.country_id = co.country_id
WHERE co.country_name = 'USA'
  AND comp.founding_year < 1965
ORDER BY comp.founding_year;
""", conn)

/var/folders/t2/9wrz89jj3tnbfwphbvxvcl8r0000gn/T/ipykernel_27279/3046472474.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("""


,ceo_name,company_name,founding_year
0,Mary Barra,General Motors,1908
1,Arvind Krishna,IBM,1911
2,Doug McMillon,Walmart,1962


In [62]:
#3. Which companies have CEOs who are younger than 60?
pd.read_sql("""
SELECT ce.first_name || ' ' || ce.last_name AS ceo_name,
       comp.company_name,
       (2025 - ce.birth_year) AS ceo_age
FROM companies comp
JOIN ceos ce ON comp.ceo_id = ce.ceo_id
WHERE (2025 - ce.birth_year) < 60
ORDER BY ceo_age;
""", conn)

/var/folders/t2/9wrz89jj3tnbfwphbvxvcl8r0000gn/T/ipykernel_27279/92535772.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("""


,ceo_name,company_name,ceo_age
0,Daniel Zhang,Alibaba Group,53
1,Andy Jassy,Amazon,57
2,Doug McMillon,Walmart,59


In [63]:
#4. What is the headquarter city of the company whose CEO was born in India? 
#LEFT JOIN for joining headquarter cities to headquarter states, even if they don't belong to a state
pd.read_sql("""
SELECT comp.company_name,
       hc.city_name as headquarter_city,
       COALESCE(hs.state_name,'—') AS headquarter_state,
       hco.country_name AS headquarter_country
FROM companies comp
JOIN ceos ce        ON comp.ceo_id = ce.ceo_id
JOIN cities bc      ON ce.city_id = bc.city_id
JOIN countries bco  ON bc.country_id = bco.country_id
JOIN cities hc      ON comp.hq_city_id = hc.city_id
LEFT JOIN states hs ON hc.state_id = hs.state_id
JOIN countries hco  ON hc.country_id = hco.country_id
WHERE bco.country_name = 'India';
""", conn)

/var/folders/t2/9wrz89jj3tnbfwphbvxvcl8r0000gn/T/ipykernel_27279/44731139.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("""


,company_name,headquarter_city,headquarter_state,headquarter_country
0,IBM,Armonk,New York,USA


In [64]:
#5. Which companies have CEOs who started in their positions after 2014? 
pd.read_sql("""
SELECT comp.company_name,
       ce.first_name || ' ' || ce.last_name AS ceo_name,
       ce.became_ceo_year
FROM companies comp
JOIN ceos ce ON comp.ceo_id = ce.ceo_id
WHERE ce.became_ceo_year > 2014
ORDER BY ce.became_ceo_year;
""", conn)

/var/folders/t2/9wrz89jj3tnbfwphbvxvcl8r0000gn/T/ipykernel_27279/1557724374.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("""


,company_name,ceo_name,became_ceo_year
0,Alibaba Group,Daniel Zhang,2015
1,Sony Corporation,Kenichiro Yoshida,2018
2,IBM,Arvind Krishna,2020
3,Amazon,Andy Jassy,2021
4,Samsung Electronics,Jong-hee Han,2022
